# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: RuiHan
Date: 08/21/2025

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [1]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

os.chdir(r"C:\Users\Yvaine\bootcamp_Rui_Han\homework\homework4")
RAW = pathlib.Path('data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? True


## Helpers (use or modify)

In [2]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [ ]:
SYMBOL = 'AAPL'
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))
if USE_ALPHA: #doesn't work
    url = 'https://www.alphavantage.co/query'
    params = {'function':'TIME_SERIES_DAILY_ADJUSTED','symbol':SYMBOL,'outputsize':'compact','apikey':os.getenv('ALPHAVANTAGE_API_KEY')}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    key = [k for k in js if 'Time Series' in k][0]
    df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns={'index':'date','5. adjusted close':'adj_close'})[['date','adj_close']]
    df_api['date'] = pd.to_datetime(df_api['date']); df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
else:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Adj Close']]
    df_api.columns = ['date','adj_close']

#v_api = validate(df_api, ['date','adj_close']); v_api

In [3]:
import yfinance as yf
SYMBOL = 'AAPL'
df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()
#df_api.columns = ['date','adj_close']
#v_api = validate(df_api, ['date','adj_close']); v_api

C:\Users\Yvaine\AppData\Local\Temp\ipykernel_22024\3498881255.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()
[*********************100%***********************]  1 of 1 completed


In [4]:
df_api.columns = ['date','close','high','low','open','volume']
v_api = validate(df_api, ['date','close','high','low','open','volume']); v_api

{'missing': [], 'shape': (63, 6), 'na_total': 0}

In [5]:
print(os.getcwd())
_ = save_csv(df_api.sort_values('date'), prefix='api', source='yfinance', symbol=SYMBOL)

C:\Users\Yvaine\bootcamp_Rui_Han\homework\homework4
Saved data\raw\api_source-yfinance_symbol-AAPL_20250821-091856.csv


## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [8]:
SCRAPE_URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {'User-Agent': 'AFE-Homework/1.0'}

try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')

    table = soup.find('table', {'id': 'constituents'})
    if table is None:
        raise ValueError('id=constituents table not found')

    rows = [[c.get_text(strip=True)
             for c in tr.find_all(['th', 'td'])]
            for tr in table.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)

except Exception as e:
    print('Scrape failed, using inline demo table:', e)
    html = ('<table><tr><th>Symbol</th><th>Security</th></tr>'
            '<tr><td>AAA</td><td>Demo Corp</td></tr></table>')
    soup = BeautifulSoup(html, 'html.parser')
    rows = [[c.get_text(strip=True)
             for c in tr.find_all(['th', 'td'])]
            for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)

# 如有数值列，可继续类型转换
if 'Price' in df_scrape.columns:
    df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')

v_scrape = validate(df_scrape, list(df_scrape.columns))
v_scrape

{'missing': [], 'shape': (503, 8), 'na_total': 0}

In [11]:
_ = save_csv(df_scrape, prefix='scrape', site='wikipedia', table='sp500')

Saved data\raw\scrape_site-wikipedia_table-sp500_20250821-094101.csv


## Documentation

- **API Source:**

!!doesn't work:
https://www.alphavantage.co/query  
Endpoint: `TIME_SERIES_DAILY_ADJUSTED`  
Parameters: `function=TIME_SERIES_DAILY_ADJUSTED`, `symbol=<TICKER>`, `outputsize=compact`, `apikey=<ALPHA_KEY>`

!!actually：  `yfinance.Ticker("AAPL").history(period="1y", interval="1d")`  
  (library call, no direct URL)

- **Scrape Source:**  
  https://en.wikipedia.org/wiki/List_of_S%26P_500_companies  
  Table: first `<table id="constituents">` (S&P 500 component stocks)

- **Assumptions & risks:**  
  - yfinance relies on unofficial Yahoo endpoints → endpoints or schema can change without notice.  
  - Yahoo may throttle or block repeated requests; large batches risk HTTP 429.  
  - Wikipedia DOM structure may change without notice; selector fragility on `id="constituents"`.  
  - `symbol` column assumed non-null and unique; future duplicates would break validation.

- **Confirm `.env` is not committed.**  
  ✅ `.env` listed in `.gitignore`; repository shows no committed secrets.